Package install and import

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from scipy.stats import pearsonr
from statsmodels.formula.api import ols
from sklearn.model_selection import train_test_split
import os
import pickle
import openpyxl

Loading the dataset

In [ ]:
# Đọc file data.pkl
df = pd.read_pickle('data/clean/data.pkl')
df.head()

Kiểm tra các cột số

In [ ]:
df.select_dtypes(include="number").columns

Kiểm tra các cột phân loại

In [ ]:
df.select_dtypes(include="object").columns

## Exploratory analysis and Visualization

### Visualization

In [ ]:
df = pd.read_pickle('data/clean/data.pkl')

In [ ]:
df.head()


In [ ]:
df.shape

Biểu đồ countplot và pie tần xuất các giá trị ở các cột

In [ ]:
for column in ['Tên hãng xe', 'Năm sản xuất', 'Tình trạng', 'Xuất xứ', 'Kiểu dáng','Hộp số','Màu ngoại thất', 'Màu nội thất', 'Số cửa', 'Số chỗ ngồi', 'Dẫn động', 'Loại động cơ', 'Dung tích']:
        # Tạo figure và axes cho countplot và pie chart
        fig, axs = plt.subplots(1, 2, figsize=(25, 5))

        # Vẽ countplot
        sns.countplot(x=column, data=df, ax=axs[0])
        axs[0].set_title(f'Countplot of {column}')

        # Đếm số lượng giá trị duy nhất trong cột và vẽ pie chart
        value_counts = df[column].value_counts()
        axs[1].pie(value_counts, labels=value_counts.index, autopct='%1.1f%%', startangle=180)
        axs[1].set_title(f'Pie chart of {column}')
        axs[1].axis('equal')  # Đảm bảo biểu đồ pie là hình tròn

        # Hiển thị biểu đồ
        plt.tight_layout()
        plt.show()

Bảng tần xuất các giá trị ở các cột còn lại

In [ ]:
column = ["Tỉnh"]
element_count_df = df[column].value_counts().to_frame().reset_index()
element_count_df.columns = ['Tỉnh', 'Count']
print(f"Column '{column}':")
print(element_count_df)
print()

## Checking distribution

#### Giá

In [ ]:
fig,axes=plt.subplots(1,2,figsize=(12,5))

# Distribution plot for the "Giá" column
sns.histplot(df['Giá'], kde=True, bins=10, ax=axes[0])
axes[0].set_title("Distribution of price")
axes[0].set_xlabel("Giá")
axes[0].set_ylabel("Count")

# Boxplot plot for the "Giá" column
sns.boxplot(data=df, y='Giá',ax=axes[1])
axes[1].set_title("Boxplot of price")
axes[1].set_xlabel("price")

# Adjust layout
plt.tight_layout()

# Display the subplots
plt.show()

* Phân bố giá có độ lệch dương
* Có một số ngoại lệ được quan sát thấy 

In [ ]:
# Giảm độ lớn của giá
df['Giá']=np.log(df['Giá'])

In [ ]:
fig,axes=plt.subplots(1,2,figsize=(12,6))

# Distribution plot for the "Giá" column
sns.histplot(df['Giá'], kde=True, bins=10, ax=axes[0])
axes[0].set_title("Distribution of price")
axes[0].set_xlabel("Giá")
axes[0].set_ylabel("Count")

# Boxplot plot for the "price" column
sns.boxplot(data=df, y='Giá',ax=axes[1])
axes[1].set_title("Boxplot of price")
axes[1].set_xlabel("Giá")

# Adjust layout
plt.tight_layout()

# Display the subplots
plt.show()

In [ ]:
# Chọn các cột có kiểu dữ liệu số
numeric_df = df.select_dtypes(include=['number'])

# Tính toán ma trận tương quan cho các cột số
numeric_df.corr().loc['Giá']

In [ ]:
# Tương quan
numeric_df.corr()

Biểu đồ heatmap giữa các cột số và cột giá


In [ ]:
plt.figure(figsize=(30, 20))

# Create the heatmap of the correlation matrix
sns.heatmap(numeric_df.corr(), annot=True, fmt='.2f')

# Display the plot
plt.show()

## Phân tích uni_variate cho các cột số

### Độ lệch

In [ ]:
numeric_df.skew()

* Năm sản xuất (-1.601737): Độ lệch âm mạnh, phân phối chủ yếu tập trung về phía bên trái trục x. Có thể hầu như các xe trong dữ liệu được sản xuất gần đây
* Giá (150.418760): Độ lệch dương mạnh, cho thấy phân phối chủ yếu tập trung phía bên phải trục x. Có thể hầu hết các xe có giá quá cao
* Số chỗ ngồi (6.348299): Độ lệch dương, cho thấy phân phối có đuôi dài bên phải. Có thể hầu hết các xe trong dữ liệu có số chỗ ngồi lớn
* Số cửa (-2.088292): Độ lệch âm, cho thấy phân phối chủ yếu tập trung về bên trái của trục x. Có thể hầy hết các xe có số cửa ít
* Số Km đã đi (151.900523): Độ lệch dương mạnh, cho thấy phân phối chủ yếu bên phải trục x. Có thể hầu hết các xe trong dữ liệu có số Km đã đi lớn

### Độ nhọn

In [ ]:
numeric_df.kurt()

* Năm sản xuất (3.376426): Độ nhọn dương cho thấy phân phối có đỉnh cao hơn và đuôi dài hơn phân phối chuẩn. Cho thấy một số xe được sản xuất ở các năm cụ thể có tần xuất cao hơn những năm còn lại.
* Giá (23055.802151): Độ nhọn dương mạnh, phân phối có đỉnh cao hơn và đuôi dài hơn phân phối chuẩn. Cho thấy một số xe có giá trị rất cao so với phần còn lại của dữ liệu.
* Số chỗ ngồi (99.984573): Độ nhọn dương mạnh, phân phối có đỉnh cao hơn và đuôi dài hơn phân phối chuẩn. Cho thấy một số xe có số cửa rất lớn so với phần còn lại.
* Số Km đã đi (23334.466853): Độ nhọn dương mạnh, phân phối có đỉnh cao hơn và đuôi dài hơn phân phối chuẩn. Cho thấy một số xe đã đi số Km rất lớn so với phần còn lại

In [ ]:
# Select numerical columns
numeric_columns = df.select_dtypes(include=[np.number]).columns

# Create subplots for the histograms
fig, axes = plt.subplots(nrows=len(numeric_columns), ncols=2, figsize=(12, 50))
fig.subplots_adjust(hspace=0.5)

# Generate histograms and box plots
for i, column in enumerate(numeric_columns):
    # Histogram
    sns.histplot(data=df, x=column, bins=20, color='skyblue',kde=True, ax=axes[i,0])
    axes[i, 0].set_xlabel(column)
    axes[i, 0].set_ylabel('Frequency')
    axes[i, 0].set_title('Histogram')

    # Box plot
    axes[i, 1].boxplot(df[column], vert=False)
    axes[i, 1].set_xlabel(column)
    axes[i, 1].set_title('Box Plot')

# Show the plots
plt.tight_layout()
plt.show()

## Phân tích uni_variate cho các cột phân loại

In [ ]:
cat=[]
for column in df.columns:
    if df[column].dtype== object:
        cat.append(column)
# Assuming 'cat' is a list of categorical column names
fig, axes = plt.subplots(nrows=len(cat), ncols=2, figsize=(12, 50))

for i, column in enumerate(cat):
    
    # Plot the count plot (histogram) in the first column of the current row
        sns.countplot(x=column, data=df, ax=axes[i, 0])
        axes[i, 0].set_xlabel(column)
        axes[i, 0].set_ylabel('Count')
        axes[i, 0].set_title('Countplot of ' + column)

    # Plot the pie chart in the second column of the current row using Pandas plot
        df[column].value_counts().plot(kind='pie', autopct='%.2f%%', ax=axes[i, 1])
        axes[i, 1].set_title('Distribution of ' + column)
        axes[i, 1].set_ylabel('')

# Adjust layout to prevent overlapping
plt.tight_layout()

# Display the subplots
plt.show()

### Phân tích bi-variate cho giá và cột số

In [ ]:
# Select numerical columns
numerical_columns = df.select_dtypes(include=[np.number]).columns

for column in numerical_columns:
    # Create subplots
    fig, axes = plt.subplots(1, 2, figsize=(12, 6))

    # Scatter plot using seaborn
    sns.scatterplot(x=df[column], y=df['Giá'], ax=axes[0])
    axes[0].set_xlabel(column)
    axes[0].set_ylabel('Giá')
    axes[0].set_title('Giá vs ' + column)
    axes[0].set_xticklabels(axes[0].get_xticklabels(), rotation=90, ha='right')

    # Heatmap using seaborn
    sns.heatmap(df[['Giá', column]].corr(), annot=True, fmt=".2f", cmap='coolwarm', ax=axes[1])

    # Set plot title
    plt.title("Heatmap - Giá vs " + column)

    # Display the plot
    plt.show()



In [ ]:
# Select numerical columns
numerical_columns = df.select_dtypes(include=[np.number]).columns
for column in numerical_columns:
    correlation, p_value = pearsonr(df['Giá'], df[column])
    print("Correlation with 'Giá' for column '{}' : {:.4f}".format(column, correlation))
    print("p-value for column '{}' : {:.4f}".format(column, p_value))
    print("------------------------------------------------------")

### Phân tích bi_variate cho cột giá và các cột phân loại

In [ ]:
categorical_column=[]
for column in df.columns:
    if df[column].dtype== object:
        categorical_column.append(column)
        
# Loop through each categorical column
for column in categorical_column:
    plt.figure(figsize=(20, 15))
    
    # Create a box plot
    sns.boxplot(data=df, x=column, y='Giá')
    
    # Set labels and title
    plt.xlabel(column)
    plt.ylabel('Giá')
    plt.title(f'Box Plot: {column} vs. Giá')
    
    # Rotate x-axis labels for better visibility
    plt.xticks(rotation=90)
    
    # Display the plot
    plt.show()

In [ ]:
from scipy.stats import f_oneway
import pandas as pd

# Assuming you have a DataFrame named 'df' containing the data
# and 'categorical_column' is a list of categorical columns
categorical_column=[]
for column in df.columns:
    if df[column].dtype== object:
        categorical_column.append(column)

for column in categorical_column:
    # Drop any rows with missing values
    data = df[['Giá', column]].copy()
    data.dropna(inplace=True)

    # Perform ANOVA
    groups = [data[data[column] == value]['Giá'] for value in data[column].unique()]
    f_statistic, p_value = f_oneway(*groups)

    # Create DataFrame to display results
    results = pd.DataFrame({'Group': data[column].unique(), 'F-Statistic': f_statistic, 'p-value': p_value})
    
    # Print the ANOVA table
    print(f"ANOVA Test for {column}")
    print(results.head())
    print("---------------------------")


In [ ]:
for i in range(len(categorical_column)):
    for j in range(i + 1, len(categorical_column)):
        # Tabulation
        tab = pd.crosstab(df[categorical_column[i]], df[categorical_column[j]])

        # Create subplots
        fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(20, 5))

        # Stacked Bar Plot
        tab.plot(kind='bar', stacked=True, ax=axes[0])
        axes[0].set_xlabel(categorical_column[i])
        axes[0].set_ylabel('Count')
        axes[0].set_title(f'Stacked Bar Plot of {categorical_column[i]} by {categorical_column[j]}')

        # Grouped Bar Plot
        sns.countplot(data=df, x=categorical_column[i], hue=categorical_column[j], ax=axes[1])
        axes[1].set_xlabel(categorical_column[i])
        axes[1].set_ylabel('Count')
        axes[1].set_title(f'Counts of {categorical_column[i]} by {categorical_column[j]}')

        plt.tight_layout()
        plt.show()

#### Kiểm tra thống kê

In [ ]:
cat_columns=[]
for column in df.columns:
    if df[column].dtype== object:
        cat_columns.append(column)
        
# Create an empty dictionary to store chi-square test results
chi_square_results = {}

# Tabulation and Chi-Square Test
for i in range(len(cat_columns)):
    for j in range(i + 1, len(cat_columns)):
        # Tabulation
        tab = pd.crosstab(df[cat_columns[i]], df[cat_columns[j]])
        
        # Perform chi-square test of independence
        chi2, p_value, dof, expected = stats.chi2_contingency(tab)
        
        # Store the results in the dictionary
        chi_square_results[(cat_columns[i], cat_columns[j])] = {'Chi-Square': chi2, 'P-Value': p_value}

# Print Chi-Square Test results
for pair, result in chi_square_results.items():
    print(f"Chi-Square Test of Independence for {pair}:")
    print(f"Chi-Square Statistic: {result['Chi-Square']}")
    print(f"P-Value: {result['P-Value']}")
    print("--------------------------------------------------")

 Chia tập dữ liệu huấn luyện và kiểm thử

In [ ]:
df = pd.read_pickle("data/clean/data.pkl")

In [ ]:
# Chia DataFrame thành tập dữ liệu huấn luyện và tập dữ liệu kiểm tra
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Lưu DataFrame huấn luyện vào tệp 
train_df.to_csv("data/train/data_train.csv")

# Lưu DataFrame kiểm tra vào tệp 
test_df.to_csv("data/train/data_test.csv")


In [ ]:
from scipy import stats

def handle_outliers(df, column):
    # Sử dụng Z-score để xác định outliers
    z_scores = stats.zscore(df[column])
    abs_z_scores = np.abs(z_scores)
    threshold = 3
    
    # Lọc ra các mẫu không phải là outliers
    df = df[(abs_z_scores < threshold)]
    
    return df

# Danh sách các cột cần xử lý outliers
columns_to_handle = ["Năm sản xuất", "Số Km đã đi", "Giá"]

# Áp dụng hàm xử lý outliers cho từng cột trên DataFrame gốc
for column in columns_to_handle:
    df = handle_outliers(df, column)


In [ ]:
# Chia DataFrame thành tập dữ liệu huấn luyện và tập dữ liệu kiểm tra
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Lưu DataFrame huấn luyện vào tệp 
train_df.to_csv("data/train/data_train1.csv")

# Lưu DataFrame kiểm tra vào tệp 
test_df.to_csv("data/train/data_test1.csv")
